<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

This notebook shows how to deploy a managed endpoint with custom scoring script and monitor using Watson OpenScale python sdk.

### Contents

1. <a href="#createmanagedep">Deploy the model as Managed endpoint in azure machine learning platform</a>
2. [Openscale Setup](#setup)
3. [Binding machine learning engine](#binding)
4. [Subscriptions](#subsciption)
5. [Performance monitor, scoring and payload logging](#scoring)
6. [Quality monitor and feedback logging](#feedback)
7. [Drift monitor and explainability](#monitors)

# Deploy GCR MLflow model to online endpoints with a custom environment and scoring script

## 1. Connect to Azure Machine Learning Workspace
The [workspace](https://docs.microsoft.com/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

### 1.1 Import the required libraries

In [1]:
!pip install azure-ai-ml | tail -n 1

In [2]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes
import os
import json

### 1.2 Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [3]:
# enter details of your AML workspace
subscription_id = "<EDIT THIS>"
resource_group = "<EDIT THIS>"
workspace = "<EDIT THIS>"

In [4]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

# Or if you are working in a compute instance in Azure Machine Learning, uncomment the below code

# ml_client = MLClient.from_config(DefaultAzureCredential())

## 2. Create Online Endpoint

Online endpoints are endpoints that are used for online (real-time) inferencing. Online endpoints contain deployments that are ready to receive data from clients and can send responses back in real time.

To create an online endpoint we will use `ManagedOnlineEndpoint`. This class allows user to configure the following key aspects:

- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Key-based authentication and Azure ML token-based authentication are supported. Key-based authentication doesn't expire but Azure ML token-based authentication does. Possible values are `key` or `aml_token`.
- `identity`- The managed identity configuration for accessing Azure resources for endpoint provisioning and inference.
    - `type`- The type of managed identity. Azure Machine Learning supports `system_assigned` or `user_assigned identity`.
    - `user_assigned_identities` - List (array) of fully qualified resource IDs of the user-assigned identities. This property is required is `identity.type` is user_assigned.
- `description`- Description of the endpoint.

### 2.1 Configure the endpoint

In [6]:
endpoint_name="binary-gcr-endpint-notebook"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Managed endpoint for binary german credit risk",
    auth_mode="key",
    tags={"foo": "bar"},
)

### 2.2 Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [7]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://binary-gcr-endpint-notebook.southcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://binary-gcr-endpint-notebook.southcentralus.inference.ml.azure.com/swagger.json', 'name': 'binary-gcr-endpint-notebook', 'description': 'Managed endpoint for binary german credit risk', 'tags': {'foo': 'bar'}, 'properties': {'createdBy': 'c943757b-86e7-417c-b3e4-45e2738fe59c', 'createdAt': '2024-08-08T19:01:03.715368+0000', 'lastModifiedAt': '2024-08-08T19:01:03.715368+0000', 'azureml.onlineendpointid': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/bias_explainability/providers/microsoft.machinelearningservices/workspaces/bias_dev/onlineendpoints/binary-gcr-endpint-notebook', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/providers/Microsoft.MachineLearningServices/locations/southcentralus/mfeO

## 3. Create a blue deployment

A deployment is a set of resources required for hosting the model that does the actual inferencing. We will create a deployment for our endpoint using the `ManagedOnlineDeployment` class. This class allows user to configure the following key aspects.

- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment where the model will run.
- `code_configuration` - The scoring script used to serve the model.
- `instance_type` - The VM size to use for the deployment. For the list of supported sizes, see [Managed online endpoints SKU list](https://docs.microsoft.com/azure/machine-learning/reference-managed-online-endpoints-vm-sku-list).
- `instance_count` - The number of instances to use for the deployment

### 3.1 Configure the deployment

Registering the model:

In [8]:
from azure.ai.ml.entities import ManagedOnlineDeployment

registered_model_name="<EDIT THIS>"
latest_model_version="<EDIT THIS>"

# Choose the latest version of the registered model for deployment
model = ml_client.models.get(name=registered_model_name, version=latest_model_version)

**Creating an environment to perform inference:**

- Add the path to the conda.yaml file (Can be accessed from the artifacts of the model selected)
- Use ubuntu image available in azure. Refer [Customize mlflow model deployments](https://learn.microsoft.com/en-us/azure/machine-learning/how-to-deploy-mlflow-models-online-endpoints?view=azureml-api-2&tabs=cli#customize-mlflow-model-deployments) for more details

In [9]:
environment = Environment(
    conda_file="conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04:latest",
)

Creating the deployment:

In [10]:
%%writefile custom_scoring_script.py
import json
import logging
import os
import pickle
import numpy as np
import pandas as pd

import mlflow
import mlflow.sklearn
from io import StringIO
from mlflow.pyfunc.scoring_server import infer_and_parse_json_input, predictions_to_json

def init():
    global model
    global input_schema
    # "model" is the path of the mlflow artifacts when the model was registered. For automl
    # models, this is generally "mlflow-model".
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "mlflow-model")
    model = mlflow.sklearn.load_model(model_path)

def run(input_data):
    import json
    if type(input_data) is str:
        dict_data = json.loads(input_data)
    else:
        dict_data = input_data

    data = pd.DataFrame.from_dict(dict_data['input'])
    result_probas = model.predict_proba(data)
    results = model.predict(data)

    if isinstance(results, pd.DataFrame):
        results = results.values.tolist()
    if isinstance(result_probas, pd.DataFrame):
        result_probas = result_probas.values.tolist()
    result_vector = []
    for result, result_proba in zip(results, result_probas):
        result_vector.append(
            {"Scored Labels": result, "Scored Probabilities": result_proba})

    output = {'output': result_vector}
    return json.dumps(output)

Writing custom_scoring_script.py


In [11]:
blue_deployment = ManagedOnlineDeployment(
    name="gcr-deployment-1",
    endpoint_name=endpoint_name,
    model=model,
    environment=environment,
    code_configuration=CodeConfiguration(
        code="./", scoring_script="score.py"
    ),
    instance_type="Standard_F4s_v2",
    instance_count=1,
)


### 3.2 Create the deployment

Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [12]:
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint binary-gcr-endpint-notebook exists
Uploading bibybinu (0.54 MBs): 100%|██████████| 542951/542951 [00:00<00:00, 1663732.12it/s]




...........................................

ManagedOnlineDeployment({'private_network_connection': None, 'package_model': False, 'provisioning_state': 'Succeeded', 'endpoint_name': 'binary-gcr-endpint-notebook', 'type': 'Managed', 'name': 'gcr-deployment-1', 'description': None, 'tags': {}, 'properties': {'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/providers/Microsoft.MachineLearningServices/locations/southcentralus/mfeOperationsStatus/odidp:83ebcacc-4777-42f5-b4f0-04f9aa0aaedb:27572388-01df-4a19-bc1e-bbe0d69d37d8?api-version=2023-04-01-preview'}, 'print_as_yaml': False, 'id': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourceGroups/bias_explainability/providers/Microsoft.MachineLearningServices/workspaces/bias_dev/onlineEndpoints/binary-gcr-endpint-notebook/deployments/gcr-deployment-1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlg-test/code/Users/bibybinu', 'creation_context': <azure.ai.ml._restclient.v20

In [13]:
endpoint.traffic = {blue_deployment.name: 100}
ml_client.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://binary-gcr-endpint-notebook.southcentralus.inference.ml.azure.com/score', 'openapi_uri': 'https://binary-gcr-endpint-notebook.southcentralus.inference.ml.azure.com/swagger.json', 'name': 'binary-gcr-endpint-notebook', 'description': 'Managed endpoint for binary german credit risk', 'tags': {'foo': 'bar'}, 'properties': {'createdBy': 'c943757b-86e7-417c-b3e4-45e2738fe59c', 'createdAt': '2024-08-08T19:01:03.715368+0000', 'lastModifiedAt': '2024-08-08T19:01:03.715368+0000', 'azureml.onlineendpointid': '/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/resourcegroups/bias_explainability/providers/microsoft.machinelearningservices/workspaces/bias_dev/onlineendpoints/binary-gcr-endpint-notebook', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/744bca72-2299-451c-b682-ed6fb75fb671/providers/Microsoft.MachineLearningServices/locations/southcentralus/mfeO

## 4. Test the deployment

Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the invoke command with the following parameters:

- `endpoint_name` - Name of the endpoint
- `request_file` - File with request data
- `deployment_name` - Name of the specific deployment to test in an endpoint

We will send a sample request using a [sample-request-sklearn-custom.json](sample-request-sklearn-custom.json) file.

In [14]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name="gcr-deployment-1",
    request_file="scoring_payload.json",
)

'"{\\"output\\": [{\\"Scored Labels\\": \\"No Risk\\", \\"Scored Probabilities\\": [0.7951253652572632, 0.2048746645450592]}]}"'

## 5. Get endpoint details

In [15]:
# Get the details for online endpoint
endpoint = ml_client.online_endpoints.get(name=endpoint_name)

# existing traffic details
print(endpoint.traffic)

# Get the scoring URI
print(endpoint.scoring_uri)

{'gcr-deployment-1': 100}
https://binary-gcr-endpint-notebook.southcentralus.inference.ml.azure.com/score


# Configure Azure managed endpoints with Openscale - Azure Machine Learning service engine

This notebook shows how to monitor the designer based endpoints deployed on Azure Machine Learning Studio using Watson OpenScale python sdk.

## 1. Setup

In [16]:
!pip install --upgrade --upgrade ibm-watson-openscale==3.0.39 --no-cache | tail -n 1

**Action:** Restart the kernel (Kernel->Restart)

#### ACTION: Get Watson OpenScale `apikey`

How to install IBM Cloud (bluemix) console: [instruction](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use)

How to get api key using bluemix console:
```
bx login --sso
bx iam api-key-create 'my_key'
```

In [17]:
CLOUD_API_KEY = '<EDIT THIS>'

In [18]:
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator,BearerTokenAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *

authenticator = IAMAuthenticator(apikey=CLOUD_API_KEY)
#authenticator = BearerTokenAuthenticator(bearer_token=IAM_TOKEN) ## uncomment if using IAM token

wos_client = APIClient(authenticator=authenticator)

wos_client.version



'3.0.39'

#### Download and preview training data set

In [19]:
!rm -rf credit_risk_training.csv
!wget "https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/credit_risk_training.csv"

In [20]:
import pandas as pd
import json

training_data_pd = pd.read_csv(
        "credit_risk_training.csv",
        dtype={'LoanDuration': int, 'LoanAmount': int, 'InstallmentPercent': int, 'CurrentResidenceDuration': int, 'Age': int, 'ExistingCreditsCount': int, 'Dependents': int})

training_data_pd.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


## Save training data to IBM Cloud Object Storage

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit getting started with COS tutorial. You can find COS_API_KEY_ID and COS_RESOURCE_CRN variables in Service Credentials in menu of your COS instance. Used COS Service Credentials must be created with Role parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription. COS_ENDPOINT variable can be found in Endpoint field of the menu.

In [24]:
IAM_URL="https://iam.cloud.ibm.com/identity/token"

COS_API_KEY_ID = "<EDIT THIS>"
COS_RESOURCE_CRN = "<EDIT THIS>" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "<EDIT THIS>" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints

BUCKET_NAME = "<EDIT THIS>" #example: "credit-risk-training-data"
training_data_file_name="credit_risk_training.csv"

In [25]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [26]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

### DataMart setup

In [28]:
wos_client.data_marts.show()

None,None,True,active,2023-12-15 02:33:56.440000+00:00,da5d7981-0363-4155-a373-70edcf4bda6f


In [29]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart da5d7981-0363-4155-a373-70edcf4bda6f


<a id="binding"></a>
## 2. Bind machine learning engines

### Bind  `Azure` machine learning studio

Provide credentials using following fields:
- `client_id`
- `client_secret`
- `subscription_id`
- `tenant`

In [32]:
AZURE_ENGINE_CREDENTIALS = {
   "client_id": "<EDIT THIS>",
    "client_secret": "<EDIT THIS>",
    "subscription_id": "<EDIT THIS>",
    "tenant": "<EDIT THIS>"
}

In [34]:
SERVICE_PROVIDER_NAME = "Azure Service"
SERVICE_PROVIDER_DESCRIPTION = "Added by Azure service tutorial WOS notebook."

In [35]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

Deleted existing service_provider for WML instance: 65d33b89-6823-4062-9407-6cae33f668c3


In [36]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.AZURE_MACHINE_LEARNING_SERVICE,
        operational_space_id = "production",
        credentials=AzureCredentials(
            subscription_id= AZURE_ENGINE_CREDENTIALS['subscription_id'], 
            client_id = AZURE_ENGINE_CREDENTIALS['client_id'], 
            client_secret= AZURE_ENGINE_CREDENTIALS['client_secret'],
            tenant = AZURE_ENGINE_CREDENTIALS['tenant']
        ),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider 653a6e1b-c8c8-4b60-801b-974aa7b23bf8 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [37]:
wos_client.service_providers.show(limit=1)

### uncomment below code to get the details of the service provider
# print(wos_client.service_providers.get(service_provider_id=service_provider_id).result)

None,active,Azure Service,azure_machine_learning_service,2024-08-19 18:20:40.053000+00:00,653a6e1b-c8c8-4b60-801b-974aa7b23bf8


Note: First 1 records were displayed.


In [39]:
asset_deployment_details = wos_client.service_providers.list_assets(data_mart_id=data_mart_id, service_provider_id=service_provider_id).result

In [40]:
print(json.dumps(asset_deployment_details,indent=2))

In [42]:
## select your model deployment GUID and get model asset details

deployment_id='<<EDIT THIS>>'
for model_asset_details in asset_deployment_details['resources']:
    if model_asset_details['metadata']['guid']==deployment_id:
        break

### Sample Scoring

In [45]:
import requests
import time
import json

scoring_url = model_asset_details['entity']['scoring_endpoint']['url']

data = {
          "input": [
            {
              "CheckingStatus": "0_to_200",
              "LoanDuration": 31,
              "CreditHistory": "credits_paid_to_date",
              "LoanPurpose": "other",
              "LoanAmount": 1889,
              "ExistingSavings": "100_to_500",
              "EmploymentDuration": "less_1",
              "InstallmentPercent": 3,
              "Sex": "female",
              "OthersOnLoan": "none",
              "CurrentResidenceDuration": 3,
              "OwnsProperty": "savings_insurance",
              "Age": 32,
              "InstallmentPlans": "none",
              "Housing": "own",
              "ExistingCreditsCount": 1,
              "Job": "skilled",
              "Dependents": 1,
              "Telephone": "none",
              "ForeignWorker": "yes"
            }
          ]
        }

body = str.encode(json.dumps(data))


token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = model_asset_details['entity']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

"{\"output\": [{\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.7951253652572632, 0.2048746645450592]}]}"


## 3. Subscription

In [47]:
feature_columns = ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker']
categorical_columns = ['CheckingStatus', 'CreditHistory', 'LoanPurpose', 'ExistingSavings', 'EmploymentDuration', 'Sex', 'OthersOnLoan', 'OwnsProperty', 'InstallmentPlans', 'Housing', 'Job', 'Telephone', 'ForeignWorker']

In [48]:
azure_asset = Asset(
            asset_id=model_asset_details["entity"]["asset"]["asset_id"],
            name=model_asset_details["entity"]["asset"]["name"],
            url=model_asset_details["entity"]["asset"]["url"],
            asset_type=AssetTypes.MODEL,
            input_data_type=InputDataType.STRUCTURED,
            problem_type=ProblemType.BINARY_CLASSIFICATION
        )

In [49]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import ScoringEndpointRequest,ScoringEndpointCredentialsAzureScoringEndpointCredentials
deployment_scoring_endpoint = model_asset_details['entity']['scoring_endpoint']

scoring_endpoint = ScoringEndpointRequest(url = model_asset_details['entity']['scoring_endpoint']['url']
                                          ,request_headers = model_asset_details['entity']['scoring_endpoint']['request_headers'],
                                          credentials= ScoringEndpointCredentialsAzureScoringEndpointCredentials(model_asset_details['entity']['scoring_endpoint']['credentials']['token']) )


deployment = AssetDeploymentRequest(
    deployment_id=model_asset_details['metadata']['guid'],
    url=model_asset_details['metadata']['url'],
    name=model_asset_details['entity']['name'],
    deployment_type=model_asset_details['entity']['type'],
    scoring_endpoint = scoring_endpoint
) 

In [50]:
training_data_reference = TrainingDataReference(type='cos',
                                              location=COSTrainingDataReferenceLocation(bucket = BUCKET_NAME,
                                                                                        file_name = training_data_file_name),
                                              connection=COSTrainingDataReferenceConnection(
                                                                        resource_instance_id= COS_RESOURCE_CRN,
                                                                        url= COS_ENDPOINT,
                                                                        api_key= COS_API_KEY_ID,
                                                                        iam_url=IAM_URL)
                                               )

In [52]:
asset_properties = AssetPropertiesRequest(
        label_column="Risk",
        prediction_field='Scored Labels',
        probability_fields=['Scored Probabilities'],
        training_data_reference=training_data_reference,
        training_data_schema=None,
        input_data_schema=None,
        output_data_schema=None,
    )


In [53]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=azure_asset,
        training_data_reference=training_data_reference,
        deployment=deployment,
        asset_properties=asset_properties,
        background_mode=False
).result
subscription_id = subscription_details.metadata.id
subscription_id




 Waiting for end of adding subscription 6039a35b-1f92-4dda-b8a0-fc8d8e8662a3 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------




'6039a35b-1f92-4dda-b8a0-fc8d8e8662a3'

List the subscription in the service provided created

In [55]:
wos_client.subscriptions.show(service_provider_id=service_provider_id)

## uncomment below code to get the details of the subscription
# print(wos_client.subscriptions.get(subscription_id=subscription_id).result)

f5ac576aad20d0ce237ff940bbe0dd2a,model,binary-german-credit-risk,da5d7981-0363-4155-a373-70edcf4bda6f,957d63a12e70eb90eb4b5bbe78c2c562,binary-german-credit-risk,653a6e1b-c8c8-4b60-801b-974aa7b23bf8,active,2024-08-19 18:29:08.334000+00:00,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3


<a id="scoring"></a>
## 4. Scoring and payload logging

### Score the credit risk model and measure response time

In [57]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id: ", payload_data_set_id)

Payload data set id:  28b9eb61-2e68-4568-9ba6-eca6b4113f75


In [58]:
import requests
import time
import json

subscription_details=wos_client.subscriptions.get(subscription_id).result.to_dict()
scoring_url = subscription_details['entity']['deployment']['scoring_endpoint']['url']

data = {
            "input": training_data_pd.sample(2).drop(['Risk'],axis=1).to_dict('records'),
        }

body = str.encode(json.dumps(data))

token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = subscription_details['entity']['deployment']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

"{\"output\": [{\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.7240934371948242, 0.27590659260749817]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.9423964619636536, 0.057603538036346436]}]}"


### Functions to adjust payload data format for OpenScale 

In [60]:
def convert_user_input_2_openscale(input_data):
    users_records = input_data['input']
    openscale_fields = list(users_records[0])
    openscale_values = [[rec[k] for k in openscale_fields] for rec in users_records] 

    return {'fields':openscale_fields, 'values':openscale_values}

In [61]:
import numpy as np
def convert_user_output_2_openscale(output_data):
    output_data=json.loads(output_data)
    users_records = output_data['output']
    openscale_fields = ["Scored Labels","Scored Probabilities"]
    openscale_values = []
    for record in users_records:
        openscale_values.append([record['Scored Labels'],record['Scored Probabilities'][0]])
    return {'fields':openscale_fields, 'values':openscale_values}

In [62]:
openscale_input=convert_user_input_2_openscale(data)
openscale_output=convert_user_output_2_openscale(result)

### Store the request and response in payload logging table

In [64]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=openscale_input,
           response=openscale_output,
           response_time=460
)])
time.sleep(10)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....
Number of records in the payload logging table: 2


In [65]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

None,1981,df5ace73-bf8e-4a99-a683-879b38ec9a08-1,1,less_100,2024-08-19T18:29:36.936Z,0.9274464249610901,"[0.9274464249610901, 0.0725535750389099]",0_to_200,furniture,0.9274464249610901,skilled,stores,none,10,19,credits_paid_to_date,1,yes,rent,1,957d63a12e70eb90eb4b5bbe78c2c562,none,No Risk,female,1,real_estate,1_to_4
None,1060,df5ace73-bf8e-4a99-a683-879b38ec9a08-2,2,100_to_500,2024-08-19T18:29:36.936Z,0.7047558426856995,"[0.7047558426856995, 0.2952441573143005]",0_to_200,radio_tv,0.7047558426856995,skilled,none,none,23,44,prior_payments_delayed,2,yes,free,1,957d63a12e70eb90eb4b5bbe78c2c562,yes,No Risk,male,1,car_other,4_to_7


<a id="feedback"></a>
## 5. Feedback logging & quality (accuracy) monitoring

### Enable quality monitoring

In [66]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 10
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [67]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224'

### Feedback records logging
Feedback records are used to evaluate your model. The predicted values are compared to real values (feedback records).

In [69]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json

### Get feedback logging dataset ID

In [70]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
#print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

In [72]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)




 Waiting for end of storing records with request id: b5445dfb-9b47-4e90-976c-7d615bdbf5bc 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [73]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [74]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run b7a3a7e2-424e-4851-bd08-88b184d3b98f 




finished

---------------------------
 Successfully finished run 
---------------------------




In [75]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-08-19 18:31:35.386000+00:00,true_positive_rate,af538176-9a8d-4af9-b019-309d7f08ab0d,0.9692307692307692,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,area_under_roc,af538176-9a8d-4af9-b019-309d7f08ab0d,0.6815850815850816,0.8,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,precision,af538176-9a8d-4af9-b019-309d7f08ab0d,0.7590361445783133,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,matthews_correlation_coefficient,af538176-9a8d-4af9-b019-309d7f08ab0d,0.4766933941810702,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,f1_measure,af538176-9a8d-4af9-b019-309d7f08ab0d,0.8513513513513513,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,accuracy,af538176-9a8d-4af9-b019-309d7f08ab0d,0.7755102040816326,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,label_skew,af538176-9a8d-4af9-b019-309d7f08ab0d,0.690933627340049,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,gini_coefficient,af538176-9a8d-4af9-b019-309d7f08ab0d,0.36317016317016315,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,false_positive_rate,af538176-9a8d-4af9-b019-309d7f08ab0d,0.6060606060606061,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-19 18:31:35.386000+00:00,area_under_pr,af538176-9a8d-4af9-b019-309d7f08ab0d,0.7575627470636076,None,None,['model_type:original'],quality,5fa6c5fd-75af-44ff-a8fe-f1d5f9ea5224,b7a3a7e2-424e-4851-bd08-88b184d3b98f,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3


Note: First 10 records were displayed.


# 7. Drift monitoring and explanations <a id="monitors"></a>

### Explainability configuration

In [76]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result




 Waiting for end of monitor instance creation ddceace0-afce-49d3-9c66-698b0d6db26b




preparing.
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


In [77]:
explainability_monitor_id = explainability_details.metadata.id
explainability_monitor_id

'ddceace0-afce-49d3-9c66-698b0d6db26b'

### Run explanation

In [80]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

None,626,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-1,2,500_to_1000,2024-08-19T18:42:50.903Z,0.920011043548584,"[0.920011043548584, 0.079988956451416]",less_0,furniture,0.920011043548584,skilled,none,none,17,28,credits_paid_to_date,2,yes,rent,1,957d63a12e70eb90eb4b5bbe78c2c562,none,No Risk,male,1,car_other,less_1
None,250,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-10,2,less_100,2024-08-19T18:42:50.903Z,0.9511843919754028,"[0.9511843919754028, 0.0488156080245972]",less_0,furniture,0.9511843919754028,skilled,stores,none,10,23,prior_payments_delayed,1,yes,rent,1,957d63a12e70eb90eb4b5bbe78c2c562,none,No Risk,male,1,real_estate,less_1
None,2476,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-2,3,less_100,2024-08-19T18:42:50.903Z,0.8365631103515625,"[0.8365631103515625, 0.1634368896484375]",0_to_200,furniture,0.8365631103515625,skilled,none,none,14,28,credits_paid_to_date,3,yes,own,2,957d63a12e70eb90eb4b5bbe78c2c562,yes,No Risk,male,2,real_estate,1_to_4
None,4131,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-3,4,100_to_500,2024-08-19T18:42:50.903Z,0.5958502292633057,"[0.5958502292633057, 0.4041497707366943]",less_0,appliances,0.5958502292633057,unskilled,bank,none,39,29,outstanding_credit,4,yes,own,1,957d63a12e70eb90eb4b5bbe78c2c562,yes,No Risk,male,1,unknown,1_to_4
None,3643,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-4,4,greater_1000,2024-08-19T18:42:50.903Z,0.5111302137374878,"[0.4888697862625122, 0.5111302137374878]",no_checking,repairs,0.4888697862625122,skilled,none,none,45,34,outstanding_credit,5,yes,own,2,957d63a12e70eb90eb4b5bbe78c2c562,none,Risk,male,1,savings_insurance,4_to_7
None,3243,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-5,3,less_100,2024-08-19T18:42:50.903Z,0.6805847883224487,"[0.6805847883224487, 0.3194152116775513]",less_0,car_used,0.6805847883224487,skilled,none,none,26,36,credits_paid_to_date,3,yes,own,1,957d63a12e70eb90eb4b5bbe78c2c562,none,No Risk,male,1,car_other,1_to_4
None,5375,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-6,4,500_to_1000,2024-08-19T18:42:50.903Z,0.8744866847991943,"[0.12551331520080566, 0.8744866847991943]",no_checking,radio_tv,0.12551331520080566,unskilled,stores,co-applicant,29,59,outstanding_credit,3,yes,free,2,957d63a12e70eb90eb4b5bbe78c2c562,yes,Risk,male,1,unknown,greater_7
None,5866,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-7,4,500_to_1000,2024-08-19T18:42:50.903Z,0.5357937812805176,"[0.5357937812805176, 0.4642062187194824]",no_checking,appliances,0.5357937812805176,skilled,none,none,33,41,prior_payments_delayed,4,yes,free,3,957d63a12e70eb90eb4b5bbe78c2c562,yes,No Risk,male,2,car_other,4_to_7
None,546,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-8,2,less_100,2024-08-19T18:42:50.903Z,0.951320230960846,"[0.951320230960846, 0.048679769039154]",0_to_200,car_used,0.951320230960846,management_self-employed,none,none,4,26,all_credits_paid_back,1,yes,own,1,957d63a12e70eb90eb4b5bbe78c2c562,none,No Risk,female,1,real_estate,less_1
None,1355,9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-9,2,less_100,2024-08-19T18:42:50.903Z,0.7092019319534302,"[0.7092019319534302, 0.2907980680465698]",0_to_200,radio_tv,0.7092019319534302,skilled,none,none,24,19,credits_paid_to_date,3,yes,own,1,957d63a12e70eb90eb4b5bbe78c2c562,none,No Risk,male,1,savings_insurance,less_1


In [84]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
print(result)
explanation_task_id=result.to_dict()['metadata']['explanation_task_ids'][0]

Running explanations on scoring IDs: ['9885fa6b-5c1c-421f-8ceb-70bdbe0d72e2-1']
{
  "metadata": {
    "explanation_task_ids": [
      "a1f2f9b9-1771-4f1f-890b-b5e8924f9343"
    ],
    "created_by": "IBMid-666003NI0G",
    "created_at": "2024-08-20T01:05:23.342399Z"
  }
}


In [88]:
wos_client.data_sets.show_records(data_set_id=payload_data_set_id)

In [89]:
import requests
import time
import json

scoring_url = subscription_details['entity']['deployment']['scoring_endpoint']['url']

payload_data = {
            "input": 
                    training_data_pd.sample(10).drop(['Risk'],axis=1).to_dict('records'),
        }

pl_body = str.encode(json.dumps(payload_data))


token = model_asset_details['entity']['scoring_endpoint']['credentials']['token']
headers = subscription_details['entity']['deployment']['scoring_endpoint']['request_headers']
headers['Authorization'] = ('Bearer ' + token)

start_time = time.time()
response = requests.post(url=scoring_url, data=pl_body, headers=headers)
response_time = int(time.time() - start_time)*1000
result = response.json()

print(json.dumps(result, indent=2))

"{\"output\": [{\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.5055540800094604, 0.49444594979286194]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.9287057518959045, 0.07129423320293427]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.9513672590255737, 0.048632752150297165]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.8164966702461243, 0.18350332975387573]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.7056454420089722, 0.29435455799102783]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.5642480850219727, 0.43575191497802734]}, {\"Scored Labels\": \"Risk\", \"Scored Probabilities\": [0.18871283531188965, 0.8112871646881104]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.6658697128295898, 0.33413028717041016]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [0.9089884757995605, 0.09101152420043945]}, {\"Scored Labels\": \"No Risk\", \"Scored Probabilities\": [

In [90]:
openscale_paylod_input=convert_user_input_2_openscale(payload_data)
openscale_payload_output=convert_user_output_2_openscale(result)

In [93]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord

print("Performing explicit payload logging.....")
wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
           scoring_id=str(uuid.uuid4()),
           request=openscale_paylod_input,
           response=openscale_payload_output,
           response_time=460
)])
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))

Performing explicit payload logging.....
Number of records in the payload logging table: 22


In [96]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift_v2" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [97]:
# Setting up the features and its importances
import random
feature_importances = [random.random() for _ in feature_columns]

# Create the dictionary
feature_importance_dict = dict(zip(feature_columns, feature_importances))
print(feature_importance_dict)

{'CheckingStatus': 0.12077687935313486, 'LoanDuration': 0.4862844694569116, 'CreditHistory': 0.9140352698180274, 'LoanPurpose': 0.1581220027836978, 'LoanAmount': 0.1554882147348282, 'ExistingSavings': 0.5588505770683856, 'EmploymentDuration': 0.8341504367254685, 'InstallmentPercent': 0.9788834254828491, 'Sex': 0.40976221239238164, 'OthersOnLoan': 0.5981423101319839, 'CurrentResidenceDuration': 0.5852946801422452, 'OwnsProperty': 0.4791434713335374, 'Age': 0.2609681411545819, 'InstallmentPlans': 0.3154679912621383, 'Housing': 0.624027919503637, 'ExistingCreditsCount': 0.8070870800378268, 'Job': 0.20134441854401408, 'Dependents': 0.0606629126071655, 'Telephone': 0.13178995963587792, 'ForeignWorker': 0.8351050878586475}


In [98]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
        "min_samples": 10,
        "max_samples": 1000,
        "train_archive": True,
        "features": {
            "fields": feature_columns,
            "importances": feature_importance_dict
        }
    }
drift_v2_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT_V2.ID,
    target=target,
    parameters=parameters
).result

drift_v2_monitor_instance_id = drift_v2_monitor_details.metadata.id
drift_v2_monitor_instance_id

'8f414fd6-3d51-4fe9-b32a-c0609ee15ecb'

### Run Drift monitor on demand

In [99]:
time.sleep(45)
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_v2_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run 343f6b2e-933d-4b73-9ccf-a186299d3f45




running........................
finished

---------------------------
 Successfully finished run 
---------------------------



In [102]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_v2_monitor_instance_id)

2024-08-20 04:16:05.633343+00:00,records_processed,8fc0ce36-7880-428d-82f4-9942bbbcee57,10.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:05.633343+00:00,confidence_drift_score,8fc0ce36-7880-428d-82f4-9942bbbcee57,0.4279,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:05.633343+00:00,records_processed,8fc0ce36-7880-428d-82f4-9942bbbcee57,10.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:05.633343+00:00,confidence_drift_score,8fc0ce36-7880-428d-82f4-9942bbbcee57,0.4282,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:11.190640+00:00,records_processed,016724c5-80a1-4b6b-a778-2ac1178f57bf,10.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:11.190640+00:00,confidence_drift_score,016724c5-80a1-4b6b-a778-2ac1178f57bf,0.4417,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:11.190640+00:00,records_processed,016724c5-80a1-4b6b-a778-2ac1178f57bf,10.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:11.190640+00:00,confidence_drift_score,016724c5-80a1-4b6b-a778-2ac1178f57bf,0.4423,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:17.810159+00:00,records_processed,f70e2442-8322-4be9-93e0-ccafab8d74fe,10.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:Scored Labels']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3
2024-08-20 04:16:17.810159+00:00,prediction_drift_score,f70e2442-8322-4be9-93e0-ccafab8d74fe,0.0616,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:Scored Labels']",drift_v2,8f414fd6-3d51-4fe9-b32a-c0609ee15ecb,13633d9b-918b-4bf4-b82e-b4a2b569fe35,subscription,6039a35b-1f92-4dda-b8a0-fc8d8e8662a3


Note: First 10 records were displayed.


## Congratulations!

You have finished the tutorial for IBM Watson OpenScale and Azure Machine Learning Studio. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

